In [91]:
import konlpy
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [92]:
print("보그체로 변환하고 싶은 한국어 텍스트를 입력하시오")
Korean_Text_b = "맛있는 김치를 만들어 보았습니다."
print(Korean_Text_b)

보그체로 변환하고 싶은 한국어 텍스트를 입력하시오
맛있는 김치를 만들어 보았습니다.


In [93]:
Korean_Text = Korean_Text_b.split(sep=' ')
Korean_Text

['맛있는', '김치를', '만들어', '보았습니다.']

In [94]:
Toknize_Okt = konlpy.tag.Okt()
Toknize_Hannanum = konlpy.tag.Hannanum()
Toknize_Kkma = konlpy.tag.Kkma()

Toknized_Korean_Okt = []
Toknized_Korean_Hannanum = []
Toknized_Korean_Kkma = []


for i in range(len(Korean_Text)):
    Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))
    Toknized_Korean_Hannanum.append(Toknize_Hannanum.pos(Korean_Text[i]))
    Toknized_Korean_Kkma.append(Toknize_Kkma.pos(Korean_Text[i]))

    for j in range(len(Toknized_Korean_Okt[i])):
        Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    for j in range(len(Toknized_Korean_Hannanum[i])):
        Toknized_Korean_Hannanum[i][j] = list(Toknized_Korean_Hannanum[i][j])

    for j in range(len(Toknized_Korean_Kkma[i])):
        Toknized_Korean_Kkma[i][j] = list(Toknized_Korean_Kkma[i][j])


In [95]:
Toknized_Korean_Okt

[[['맛있는', 'Adjective']],
 [['김치', 'Noun'], ['를', 'Josa']],
 [['만들어', 'Verb']],
 [['보았습니다', 'Verb'], ['.', 'Punctuation']]]

In [96]:
Toknized_Korean_Hannanum

[[['맛있', 'P'], ['는', 'E']],
 [['김치', 'N'], ['를', 'J']],
 [['만들', 'P'], ['어', 'E']],
 [['보', 'P'], ['아ㅂ니다', 'E'], ['.', 'S']]]

In [97]:
Toknized_Korean_Kkma

[[['맛있', 'VA'], ['는', 'ETD']],
 [['김치', 'NNG'], ['를', 'JKO']],
 [['만들', 'VV'], ['어', 'ECD']],
 [['보', 'VV'], ['았', 'EPT'], ['습니다', 'EFN'], ['.', 'SF']]]

In [98]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [99]:
ends_with_jongsung("ㄴ")

True

In [100]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [101]:
def guanhyeongsa_eomi_change(letter, letter2): # 관형사 어미 알맞게 change
    '''
    if letter2 == "ㅇ"
    guanhyeonsa_eomi = "하는" if ends_with_jongsung(letter) else "는" # 받침 있고, 받침 없고
    guanhyeonsa_eomi = "한"  if ends_with_jongsung(letter) else ""

    return guanhyeonsa_eomi
    '''
    pass

In [102]:
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


client_id = "DItIw9wyPPPRDPx4Hy46"
client_secret = "L6ZEleX9Mp"
url = "https://openapi.naver.com/v1/papago/n2mt"
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()

vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Hannanum[word][w_divided][0])
                data = "source=ko&target=en&text=" + Toknized_Text_English
                request = urllib.request.Request(url)
                request.add_header("X-Naver-Client-Id", client_id)
                request.add_header("X-Naver-Client-Secret", client_secret)
                response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                rescode = response.getcode()

                if rescode==200:
                    response_body = response.read()
                    json_word = json.loads(response_body.decode("utf-8"))
                    result = json_word['message']['result']['translatedText']
                    payload = {'input': result}
                    res = sess.post(url2, data=payload)
                    konglish = res.json()['output'].replace('?','')
                    Toknized_Korean_Kkma[word][w_divided][0] = konglish
                    print(konglish)
                    vogulized_sentence += konglish

                else:
                    print("Error")

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Kkma[word][w_divided][0])
                if Toknized_Korean_Kkma[word][w_divided][1] == 'JKS':
                    Toknized_Korean_Kkma[word][w_divided][0] = josa_change(Toknized_Korean_Kkma[word][w_divided-1][0][-1], Toknized_Korean_Kkma[word][w_divided][0])

                if Toknized_Korean_Kkma[word][w_divided][1] == 'ETD' or Toknized_Korean_Kkma[word][w_divided][1] =='EFN':
                    if Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한"
                    elif Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Kkma[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

번역 가능한 품사 단어 : 맛있
이트셀리시아우스
번역 불가능한 품사 단어 : 는
번역 가능한 품사 단어 : 김치
김치
번역 불가능한 품사 단어 : 를
번역 가능한 품사 단어 : 만들
일
번역 불가능한 품사 단어 : 어
번역 가능한 품사 단어 : 보
스텝
번역 불가능한 품사 단어 : 았
번역 불가능한 품사 단어 : 습니다
이트셀리시아우스는 김치를 일어 스텝았습니다 
맛있는 김치를 만들어 보았습니다.


In [103]:
Toknized_Korean_Kkma # 앞에 받침 있으면 하려, 없으면 려

[[['이트셀리시아우스', 'VA'], ['는', 'ETD']],
 [['김치', 'NNG'], ['를', 'JKO']],
 [['일', 'VV'], ['어', 'ECD']],
 [['스텝', 'VV'], ['았', 'EPT'], ['습니다', 'EFN'], ['.', 'SF']]]

In [104]:
translator = Translator(service_urls=[
      'translate.google.co.kr',
    ])
result = translator.translate(Korean_Text_b, src='ko', dest='en')
print(result.text)

I made a delicious kimchi.


In [105]:
Korean_Text_b

'맛있는 김치를 만들어 보았습니다.'

In [106]:
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


translator = Translator(service_urls=['translate.google.co.kr'])
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()
vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])

                konglish = translator.translate(Toknized_Korean_Hannanum[word][w_divided][0], src='ko', dest='en').text
                payload = {'input': konglish}
                res = sess.post(url2, data=payload)
                konglish = res.json()['output'].replace('?','')
                Toknized_Korean_Kkma[word][w_divided][0] = konglish
                print(konglish)

                vogulized_sentence += konglish

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Kkma[word][w_divided][0])
                if Toknized_Korean_Kkma[word][w_divided][1] == 'JKS':
                    Toknized_Korean_Kkma[word][w_divided][0] = josa_change(Toknized_Korean_Kkma[word][w_divided-1][0][-1], Toknized_Korean_Kkma[word][w_divided][0])

                if Toknized_Korean_Kkma[word][w_divided][1] == 'ETD' or Toknized_Korean_Kkma[word][w_divided][1] =='EFN':
                    if Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한"
                    elif Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Kkma[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

번역 가능한 품사 단어 : 맛있
딜리셔스
번역 불가능한 품사 단어 : 는
번역 가능한 품사 단어 : 김치
김치
번역 불가능한 품사 단어 : 를
번역 가능한 품사 단어 : 만들
메이크
번역 불가능한 품사 단어 : 어
번역 가능한 품사 단어 : 보
페이퍼
번역 불가능한 품사 단어 : 았
번역 불가능한 품사 단어 : 습니다
딜리셔스는 김치를 메이크어 페이퍼았습니다 
맛있는 김치를 만들어 보았습니다.
